In [1]:
import scipy.io as scio
import  numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpig
import pickle

In [2]:
data = scio.loadmat('data/mldata/mnist-original.mat')
data.keys()

dict_keys(['__header__', '__version__', '__globals__', 'mldata_descr_ordering', 'data', 'label'])

In [3]:
data['label'].shape, data['data'].shape

((1, 70000), (784, 70000))

In [4]:
x_train,x_test = data['data'][:,:60000].T,data['data'][:,60000:].T
x_train.shape,x_test.shape

((60000, 784), (10000, 784))

In [5]:
imag = Image.fromarray(x_test[999].reshape(28,28))
imag.show()

In [6]:
image = Image.fromarray(x_train[1].reshape(28,28))
# image.show()
image.save('data/image.png')
image

In [7]:
x = Image.open('data/image.png')
np.array(x).shape
x

In [8]:
x_train_target, x_test_target = data['label'][:,:60000].reshape(60000), data['label'][:,60000:].reshape(10000)

In [9]:
x_train_target.shape,x_test_target.shape

((60000,), (10000,))

In [10]:
with open('data/sample_weight.pkl','rb') as f:
    network = pickle.load(f)
network.keys()

dict_keys(['b2', 'W1', 'b1', 'W2', 'W3', 'b3'])

In [11]:
(network['W1'].shape,network['b1'].shape,network['W2'].shape,network['b2'].shape,
network['W3'].shape,network['b3'].shape)

((784, 50), (50,), (50, 100), (100,), (100, 10), (10,))

In [12]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

def softmax(x):
    c = np.max(x,axis=1).reshape(100,1)
    exp_a = np.exp(x-c) ##防止数据溢出
    sum_exp = np.sum(exp_a,axis=1).reshape(100,1)
    return exp_a/sum_exp

In [13]:
x_train_temp = x_train[:100]
x_train_temp.shape

(100, 784)

In [14]:
network['W1'].shape

(784, 50)

In [15]:
result = np.dot(x_train_temp,network['W1'])+network['b1']
result = sigmoid(result)
result = np.dot(result,network['W2'])+network['b2']
result = sigmoid(result)
result = np.dot(result,network['W3']) +network['b3']
result = softmax(result)
result.shape,result[0], np.sum(result[0])

d:\python\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  


((100, 10),
 array([9.99495268e-01, 5.39910339e-10, 7.97096582e-05, 8.05681793e-06,
        1.05080005e-07, 3.98143311e-04, 1.30742392e-05, 4.09953236e-06,
        7.32786930e-07, 7.32636011e-07], dtype=float32),
 0.9999999)

In [16]:
p = np.argmax(result, axis=1)
p.shape,p

((100,),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64))

In [17]:
x_target_temp = x_train_target[:100]
x_target_temp.shape
np.sum(x_target_temp == p )

100

In [18]:
def mean_square(y, t):
    return  0.5*np.sum((y-t)**2)

## 均方误差函数
## $ E =\frac{1}{2}\sum_k(y_k-t_k)^2$
### $y_k$表示神经网络的输出，$t_k$表示监督数据，k为维数
### 如果监督数据用one-hot形式表示的话，那么$y_k$，$t_k$为如下形式
### y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0, 0.1, 0, 0]
### t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]

In [19]:
# 设2 为正确解，则有下面
t = [0,0,1,0,0,0,0,0,0,0]
y = [0.1,0.05,0.6,0,0.05,0.1,0,0.1,0,0] #此时2的概率最高
mean_square(np.array(t),np.array(y))

0.09750000000000003

In [20]:
y1 = [0.1,0.05,0.1,0,0.05,0.6,0,0.1,0,0] #此时7的概率最高，但正确结果应该是2
mean_square(np.array(t),np.array(y1))
# 可以看到，均方误差函数可以用来衡量结果的准确程度

0.5975

## 交叉熵误差函数
### $ E = -\sum_k t_klogy_k$


In [21]:
def cross_entropy_error(y,t):
    delta = 1e-7
    return -np.sum(t*np.log(y+delta))

In [22]:
# 设2 为正确解，则有下面
t = [0,0,1,0,0,0,0,0,0,0]
y = [0.1,0.05,0.6,0,0.05,0.1,0,0.1,0,0] #此时2的概率最高
cross_entropy_error(np.array(t),np.array(y))

6.447238200383332

In [23]:
y1 = [0.1,0.05,0.1,0,0.05,0.6,0,0.1,0,0] #此时7的概率最高，但正确结果应该是2
cross_entropy_error(np.array(t),np.array(y1))
# 可以看到，交叉熵损失函数可以用来衡量结果的准确程度

14.50628607586249

In [24]:
def change_onehot(x):
    T = np.zeros((x.shape[0], 10))
    for idx, row in enumerate(T):
        row[int(x[idx])] = 1
        # print(idx, row, x[idx])
    return T

In [25]:
x_train_target = change_onehot(x_train_target)
x_train_target

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [26]:
x_train_target.shape,x_train_target[30000:30010]

((60000, 10),
 array([[0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]]))

In [27]:
x_train_target.size
def cross_entopy_error_super(y,t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
    batch_size = y.shape[0]
    return -np.sum(t*np.log(y+1e-7))/batch_size

In [33]:
x_train_target.shape,x_train_target.ndim,x_train_target.size


((60000, 10), 2, 600000)

In [41]:
temp = np.array([[1,2,3,4],[2,3,4,6]])
temp.size,temp.shape,

(8, (2, 4))